# UMAP chess dataset EDA

---

## Staging

---

import section:

In [22]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

import umap

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import make_column_transformer

general parameters:

In [23]:
## metadata
notebook_ver = 0.3

## dataset
data_pth = './data/games.csv'

## Define nominal data variables to be thrown at one-hot-encoder
to_hot_encode = [
    'opening_eco',
    'increment_code',
    'victory_status'
]

## Define string variables for vectorization
to_vectorize = [
    'white_id',
    'black_id'
]

## Define numerical cols to use
num_cols = [
    'winner_bin',
    'rated_bin',
    'game_len_mins',
#     'turns', # dense var
#     'white_rating', # dense var
#     'black_rating', # dense var
    'opening_ply'
]

Data load:

In [24]:
GAMES = pd.read_csv(data_pth)

## Preprocessing

---

In [25]:
## Encode binary vars to integers
GAMES['rated_bin'] = np.where(GAMES['rated'] == True, 1, 0)

GAMES.drop(columns='rated',
          inplace=True)

In [26]:
GAMES['winner_bin'] = np.where(GAMES['winner'] == 'white', 0, 1)

GAMES.drop(columns='winner',
          inplace=True)

In [27]:
## Opening, closing games dates transformation
GAMES['created_at_dt'] = pd.to_datetime(GAMES['created_at']/1000, unit='s', origin='unix')
GAMES['last_move_at_dt'] = pd.to_datetime(GAMES['last_move_at']/1000, unit='s', origin='unix')

GAMES['game_len_dt'] = GAMES['last_move_at_dt'] - GAMES['created_at_dt']
GAMES['game_len'] = GAMES['last_move_at'] - GAMES['created_at']

GAMES['game_len_mins'] = GAMES['game_len_dt'].dt.components['minutes']

## Part 1 - Metadata table

---

In [28]:
## Instantiate One Hot Encoder
ohe = OneHotEncoder(handle_unknown='error')

In [29]:
## Instantiate Count Vectorizer
cve = CountVectorizer()

In [30]:
## Set column transformer
column_trans = make_column_transformer((ohe,to_hot_encode),
                                       remainder='passthrough')

In [31]:
## Fit column transformer
column_trans.fit(GAMES[to_hot_encode])

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder', OneHotEncoder(),
                                 ['opening_eco', 'increment_code',
                                  'victory_status'])])

In [32]:
## Produce one-hot-encoded array 
hot_array = column_trans.transform(games[to_hot_encode]).toarray()

HOT_ENCODED = pd.DataFrame(hot_array, 
                              columns = column_trans.get_feature_names())

/home/nef/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [33]:
## Form final dataframe
META = GAMES[num_cols].merge(HOT_ENCODED,
                     left_index=True,
                     right_index=True)

### Metadata table

In [34]:
META

winner_bin  rated_bin  game_len_mins  opening_ply  \
0               0          0              0            5   
1               1          1              0            4   
2               0          1              0            3   
3               0          1              0            3   
4               0          1              0            5   
...           ...        ...            ...          ...   
20053           0          1              5            2   
20054           1          1             12            2   
20055           0          1              2            3   
20056           0          1             15            4   
20057           1          1             12            3   

       onehotencoder__x0_A00  onehotencoder__x0_A01  onehotencoder__x0_A02  \
0                        0.0                    0.0                    0.0   
1                        0.0                    0.0                    0.0   
2                        0.0                    0.0                    0.0   
3                        0.0                    0.0                    0.0   
4                        0.0                    0.0                    0.0   
...                      ...                    ...                    ...   
20053                    0.0                    0.0                    0.0   
20054                    0.0                    0.0                    0.0   
20055                    0.0                    0.0                    0.0   
20056                    0.0                    0.0                    0.0   
20057                    0.0                    0.0                    0.0   

       onehotencoder__x0_A03  onehotencoder__x0_A04  onehotencoder__x0_A05  \
0                        0.0                    0.0                    0.0   
1                        0.0                    0.0                    0.0   
2                        0.0                    0.0                    0.0   
3                        0.0                    0.0                    0.0   
4                        0.0                    0.0                    0.0   
...                      ...                    ...                    ...   
20053                    0.0                    0.0                    0.0   
20054                    0.0                    0.0                    0.0   
20055                    0.0                    0.0                    0.0   
20056                    0.0                    0.0                    0.0   
20057                    0.0                    0.0                    0.0   

       ...  onehotencoder__x1_9+9  onehotencoder__x1_90+0  \
0      ...                    0.0                     0.0   
1      ...                    0.0                     0.0   
2      ...                    0.0                     0.0   
3      ...                    0.0                     0.0   
4      ...                    0.0                     0.0   
...    ...                    ...                     ...   
20053  ...                    0.0                     0.0   
20054  ...                    0.0                     0.0   
20055  ...                    0.0                     0.0   
20056  ...                    0.0                     0.0   
20057  ...                    0.0                     0.0   

       onehotencoder__x1_90+120  onehotencoder__x1_90+30  \
0                           0.0                      0.0   
1                           0.0                      0.0   
2                           0.0                      0.0   
3                           0.0                      0.0   
4                           0.0                      0.0   
...                         ...                      ...   
20053                       0.0                      0.0   
20054                       0.0                      0.0   
20055                       0.0                      0.0   
20056                       0.0                      0.0   
20057              

## Part 2 - Vectorized moves table

---

In [35]:
## create a algebraic notation to iccf notation board mapping
# iccf_board = [str(x)+str(y) for x in range(1,9) for y in range(1,9)]
# algebraic_board = [l + str(i) for l in list('abcdefgh') for i in range(1,9)]
# board_mapping = {algebraic : iccf for algebraic, iccf in zip(algebraic_board, iccf_board)}

In [36]:
GAMES.columns

Index(['id', 'created_at', 'last_move_at', 'turns', 'victory_status',
       'increment_code', 'white_id', 'white_rating', 'black_id',
       'black_rating', 'moves', 'opening_eco', 'opening_name', 'opening_ply',
       'rated_bin', 'winner_bin', 'created_at_dt', 'last_move_at_dt',
       'game_len_dt', 'game_len', 'game_len_mins'],
      dtype='object')

In [37]:
def to_uni(mov):
    '''
    Turn str move encoding into a sequence of Unicode values
    '''
    move_unicode = []
    for char in mov:
        move_unicode.append(ord(char))
    return move_unicode

In [38]:
def encode_moves(moves_seq):
    '''
    Encode move into an integer = 1*e^10
    '''
    moves = []
    for move in moves_seq.split(' '):
        ## Convert str encoding to unicode vals
        move_uni = to_uni(move)
        ## Multiply all unicode vals together
        move_prod = np.prod(np.array(move_uni))  
        ## Take log2 for every move and multiply by 10_000_000
        moves.append(int(np.log2(move_prod)*10_000_000))
    return(moves)

In [39]:
## Encode moves into separate variable
GAMES['moves_encoded'] = GAMES['moves'].apply(lambda x: encode_moves(x))

In [40]:
## Check max() len of moves sequence in dataset
GAMES['moves_len'] = GAMES['moves_encoded'].apply(len)
moves_max = GAMES['moves_len'].max()

In [61]:
MOVES = GAMES[['moves_encoded','id']]

In [87]:
MOVES_EXPLODED = (MOVES
                .explode(column='moves_encoded'))

In [88]:
MOVES_EXPLODED

moves_encoded        id
0         123442959  TZJHLljE
0         123717766  TZJHLljE
0         123297963  TZJHLljE
0         123842441  TZJHLljE
0         259080238  TZJHLljE
...             ...       ...
20057     241209151  N8G2JHGG
20057     186540345  N8G2JHGG
20057     241087450  N8G2JHGG
20057     186800298  N8G2JHGG
20057     239346994  N8G2JHGG

[1212827 rows x 2 columns]

In [122]:
MOVES_EXPLODED.iloc[1946]

moves_encoded    253785975
id                mpzYDfIK
Name: 45, dtype: object

In [126]:
from tqdm import tqdm_notebook, tnrange

transposed_moves = []

for game in tnrange(0, len(MOVES_EXPLODED.index.values)):
    
    
    GRABBED_MOVES = MOVES_EXPLODED.loc[game]
    
    cols = range(1,len(GRABBED_MOVES)+1)
    game_id = GRABBED_MOVES['id']
    
    
    if type(game_id) != str:
        game_id = game_id.values[0]
    
    
    GRABBED_MOVES = GRABBED_MOVES.drop(columns='id')
    
    TRANS_MOVES = pd.DataFrame(GRABBED_MOVES).T
    
    TRANS_MOVES.index = [game_id]
    TRANS_MOVES.columns = cols

    transposed_moves.append(TRANS_MOVES)

/tmp/ipykernel_8525/3931987414.py:5: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for game in tnrange(0, len(MOVES_EXPLODED.index.values)):


  0%|          | 0/1212827 [00:00<?, ?it/s]

TZJHLljE
l1NXvwaE
mIICvQHh
kWKvrqYL
9tXo1AUZ
MsoDV9wj
qwU9rasv
RVN0N3VK
dwF3DJHO
afoMwnLg
HgKLWPsz
Vf5fKWzI
HRti5mKv
2fEjSei6
u7i6dOaJ
guanvMR5
PmpkWkmm
EwaK0IsE
yrSDozT3
x31mXlvc
oQklnWWp
QFCZwY1f
5cgBygpI
UhXXBOMY
mCij4hBq
ifUMWtVj
Wf0zuLQQ
srz9QfSN
NS6ccssb
M3vpf2Ki
fXhNOnOn
DiCTkvZH
FSkgvV2E
F1mRjzPr
Lq24M4KQ
lHNM8cl5
GfeEgLV3
OR6aULrW
Yg006qwa
G5S21pXK
R9a2DLwe
r4LcBZA7
SDDXGw21
oqoT4DLS
eSociDPG
mpzYDfIK
27zreJYy
IRYkgI8N
tLnhk43J
iq3Kz06t
AAw1TiNN
n4vRYUCW
KYPdQv8e
p30PeXvE
PBS4FEld
0lDqWVlz
O3wdIwg7
48ktrxLp
i2GBGO0R
QZSTu1aT
9tU9MM6P
ztpC1S2c
qc8aH5XL
6sTz6t7D
XuyVUkDi
D5Oa4vGO
6QAiH2Yy
l7top9ps
nG3Of11I
cUzM68JF
NZSBXWl2
DHuXFP6C
00R4UOcU
IN15cBnB
WYOts2ME
oWFQ8mnz
i2I8y9Xh
osPCRdxJ
Mi1AmkiI
7kK30TmL
ThCXomTn
4dH72B3d
4jRtIZTy
wNK2HdLb
d62Tg4dc
TaSQfvww
Xwpzq87m
AHmgnsKz
MfNml1rl
mEAvshF2
ySsVdRfw
ylLqmXwb
8mceLIXN
YmJpuJkM
PNNfempy
TVfozmF5
4LX77G3m
nBfSissu
h1unxHlo
OIyTAHCP
bnCnIJSz
ZzWyY1f8
mkKSL3Ev
aVwWNjNR
DNUmSBH4
Tlng5eqN
wljkejE0
iNZuj8bY
Kan2JKOp
bCUwPEJ7
zTwM10Si
P

B7pm4CS7
CZydGTTM
ZzbCZgxF
bslFqd73
pedSCqh0
R5eh7vmy
plyLjw3t
ZOMAHujN
uOfI8FWS
kjcAhcqA
fpy4l7uG
gyp74acJ
ZPUl9Yf4
GXfR791M
XiNBcXSZ
rygDeebM
d2LSj4j4
vd0nKjtQ
j6KBt6J0
fUXsv25b
dh097wuJ
UCMyX43Q
XrBt8qq0
v3ryBvEc
v0LKCXys
sCGHOpvl
ZERC7Ug4
UGqVq7Ch
sARMWAcM
JTF5lktc
tIbVGp2o
02V52u8v
U3GLEkAY
2NE5eGaX
A2Owhl4L
73yeVDmb
SnAAGIyU
nfx9Gu0l
ehEMiCKy
fv9uCJeh
6zZ47KG5
9BUi3r5Q
csF2CjMH
VLDmS9nM
M3C2yjzI
hyCmLj7A
KCRA9rHk
KrAwDchk
UfgTlFo1
9N9o9zK6
8mE1pMht
wV77QKJh
Nrwnvyrx
tyWYBpMs
zPwZcNlm
lIsZEF0G
WuGtllvl
UKdc4gix
K7MB0oNd
MBnszsWx
kWXQ0xsQ
b1MtlubX
hXMTCGhS
DyOgBxNv
C93FmFIq
8xdSsaAF
Zv9RJQGg
RsERFEbb
i9ZGHc3z
RIEBee5J
O8dHQVTA
A0xmkXW2
b1MLIFpL
vmVeJL73
a5y8QJBk
FeCXlUdI
PStuE3Ru
4pZgsdZ6
NNDUMnRw
FyP8fltG
ahCi7B8k
pVodVKhK
dnwPq0Ru
rohKfyT8
xHL05fIw
lLMchIam
OuFnfDxZ
Ju3yDDDT
ocEoaR9Q
FZByGsHT
8lqflAf6
hq3TSGej
43etu8Sq
xPLiusuQ
JlmtraYB
YAeg03yO
5qXqLPDB
q8gzJMaa
CUpTQECd
MO9Hhejy
ZkzVaz0B
OJdeMip2
anSWcBOw
iYVm4ANx
khJOZetm
8CCBHpef
MP8TYTbX
mu5KYglJ
2shZQ2X6
4qRYXCHX
XNMbvz8v
I

XwXKKRgH
uPzqP2FV
tcldEfFZ
Pzz9Gc9Q
ewxjkJOT
tWOp5Xrm
186UEYVk
stvJGhal
82ivBVh7
kJwtZMT4
akxVQ1pj
jbcrVMhU
UabINQ9F
FC1RzqO0
NsDNnKKA
nAFxszYC
NUyvOcew
6lJ8uXQ9
0DfCvanp
KtIBNyPW
jmZDoRr6
d1YBFVRp
sGS3Ggr9
EwxVkyW0
gvGYffbv
kvdSJ1Uv
hquBhRBh
mzbjFWVu
0fg2R7L5
c0fNR2oA
FIolUvJT
FFYyj8Er
MbpgeF2I
ut61FwwT
KAnW0Zjf
mOttSkCs
3K5kYPO8
rsgQk480
35KwPzRM
eKq8d8ce
IBag6UPi
9328ZKby
BhJuKOZS
CrQ4HxKE
heaDbzar
OyDuJ1Cz
XJ9s4FNH
joGFmaEx
YI9iirSM
OyQkHBIB
tYjmcbT8
8c1sF2CM
wX3EQAPu
rVHJzpo8
LtakuRwC
yPT5JaJx
dv89jgip
uhsgwEWX
qt1farO7
3jaOrAPh
2Oki1TZT
Ry7WqOgZ
Z7JqMQzu
9MpjI8xF
b5lyUYiv
kmWD4yQ1
CFPz3QHX
NfBNi0eX
de8kkad4
yvqpdUQE
dQCCZY4f
8zGwL053
2QYrX7bz
QGXLgc6L
J62F4oVs
M1zbHN7r
i4dYtT5Z
hjDUxNxI
L0ZvDNsO
D4yJepcs
9sVsFP7b
MAf9hU7e
zmE33vKV
wNCZx1CT
P52dSnkE
01SY3OrQ
glJoDC2y
lM6NQlk8
D9ojmq85
xhjvReVc
6nkIRlhw
X1utMl0m
ALSAOdUO
EN6icCh9
cTVwQAiH
Mixsmun3
f8Irc42M
WnW6BkKp
NUxCVtS8
zHLyaVIM
Wjc2TBuJ
HwhSbsZW
cYQNRZU8
C9t9CsHd
rVNCsu7X
ltNvi8Sy
UHUPy1Q3
zKnI08sI
mG08UIle
A1hhaRTf
ypgpDIf0
U

...

k9C46bKy
f6G2gege
cXPlDJbB
C3yXB6tN
Lhso0TKA
NQVfmGpB
2MZgnZTe
CeQEZAsj
YHdbw2oT
F8meVYY5
79DVKPvm
FDzRtaH1
Y3RL2izC
H2kDETrs
IjB9SzbU
jRvDxm9x
uOx3YYrc
bRkJWIYs
ML1azgpb
VBJxhSUo
1Uz1cIyf
bIEBCC9G
ApBNB83G
a8qqE4lg
m2PK0MFq
KmGbWRAu
NmJ79CvK
GiaLTHMH
NhGYnJFZ
qndc4Gua
orKxScLk
mFGYvAie
YKbY0Vsc
d6Iu0HW2
fPs2gdTz
YFh1OkAd
u1vZFHlL
1UtoXhHJ
bjpL1ZQZ
r147vt7e
xqz58g8e
oqzwups7
thowfqmx
wr2hpxd0
12uzibn4
eavyc681
07qtttdh
6bafjkfg
ZUB9i9KD
TZZfUw7O
9213Cgol
zAUu6xjg
ffE0eoW5
Cevosy96
tMG2OmHZ
IyyzP0mj
5CXuugNv
blhg30qk
OlSbQOu6
awl2EFfF
wi5hQYbG
SaPZBsIz
XE3upCTG
FiXhsOuU
ChoZxjDQ
nJQnDe7U
zerjgddu
ZnhkvCHu
f6ZG1RO2
2RmkFRKu
ElvyqfrW
0CP83noT
2BAAOqIN
RSRm29i4
Yh8bMb0P
4c813YSv
7lyNsOKb
IWZbujcy
54T3X2uj
9SDTUNGB
QBLj8DlS
13A6xNW7
H7XA5jqJ
TDvZiP6g
n4prFhwI
n8MvWLRR
Ma3XFU1Z
oAWZTknM
LrSVml8V
VMTl0qqo
MZdxUsly
2iIwlAqq
hFygaJm8
F0tgQ8MH
AcCDBKeI
TLvyDS6E
oOX3ZSyc
qooiHedB
GgSPdAV7
TMhkCy7Z
OeouScGh
PAQUNQay
IeAl48MW
t4B5UKq4
5CwkPpHi
EBW4Mglp
5wjdYZnY
itzogi3B
NDTOYhp4
0Mfz0EA1
zTx4YImd
X

k3buQmpE
qIStgGJx
e7lWJcnK
NfyHNEiR
z6gg3YYt
w6yX99ya
un86XZiS
TaJPxLiw
r2khP5Vs
IVrAnmJv
vZvrBGAQ
9ptYOW8L
awZ3R17W
RS68WCJP
jwaipTby
u1g5LTHq
TciMcYZX
pi46u5H1
13Bhzfum
hUEg89zo
vFtmDOwT
3RyTRCMI
2n4fQAJi
bB8WETtM
Xe3gOrsz
NTp13DPP
ITipQLDu
xJDUaj1h
vWoHMisO
VhJ7cV1L
KCqT9eNw
Bs8JtkKB
6qe4OBBp
FtwV4vsK
dYbRt8WE
6fDBMspI
nCjGdt4W
3CmelaEs
Xb7CydfV
dmDRp2sC
cmfM3vr9
Hj23Y0k5
FoGBNiqO
fPQvNK6L
TPRYE54j
cLZLdARn
bbgVRZrl
299htGOa
bM6kL3YE
cqvcjaDD
ySnYe7O0
IRsVxsLY
KmqpVjte
MGkqyA4X
2Nk1rUCU
kzxfItmB
vtCUw4Z9
dOQaxzEn
RjODDIbs
IDbDPz4E
Vt6ziGBL
pvahQeRF
AKuWPODR
MBtZLMaS
fsVliFww
DmFupUp9
MJDEd1R1
4bhYjzWn
07qaUepw
peDF4tDs
71X7CReI
bb1svOss
fP7jQopo
1zIBWzYk
Aub91G4K
aq5q3fp8
nHYxIoXC
fSfVNBR7
uvux5Jib
E9a3UGvL
Iv2oLvpe
WKTzddNg
nkOcFkEX
NwBz5JK1
p1oFoNas
bGiGLLkt
qUetu3Fa
LmAF9EOQ
ZWXisimD
VW8jn694
VCm6Y9sa
JfT9yIBl
1GJI8zgp
1vZFGZU4
orGL4RMu
5ORW5WNC
Bys3Uii4
6XdCEkkJ
VCvbNgko
kvaVGWlk
0gTDO6Av
t7vvcwqO
igSDFKv0
facMwkUo
j5KY62yS
nyYULwUl
h0YsGMhj
o4T7dj5h
VgNt0SSX
UvqLtnPM
1b0kpInt
d

oiiHte7u
U4BPX0RP
gvhODN4e
stZVXiNv
pEq8Wptk
Qb0cJ4h1
gg5qpgov
jE8qRfOQ
8TXODbts
QJVU7Kcg
jsSUUfbK
v2XDN50z
7hit8cPS
XVbvDgrS
UM1nnT09
yJA7BcWS
XajnTMHg
Qklh7FBI
N9q5r2gD
VjrTwpKb
yWK4N1Gs
6mELPaah
eP09Yv7W
KfuKdhkr
V7a3QIoB
mzWzFRLc
eBQhNVnO
pSa9EQDs
2QfmNLYQ
S9bl3cZ1
ExKINFtZ
23WLlukF
DTfofNSS
6YAtxCka
JLfmEWad
FqftsOiN
mskSEbwv
1hVQBHde
MRBPa9AX
iCq06qa3
W9898sKn
cUi6SZv8
pmaYJ9Kv
dKenDQ27
sfeyoLzl
79RGpjMH
KVgN96ck
65Um9Tcf
stmk2OCH
niNth6JB
0rISydr6
9JFLJi6k
inyTCeV2
td0tnDDQ
uMoXrR1H
RmLnL0CM
djXJIqVW
srcEDwLR
zlQThGVX
iZxEsXji
0zirgLwT
DlQedaAr
yP8CgGMt
2X95HfiC
A5nRZTL3
S9SV1nuZ
45MwnW73
VCcAlfky
FDq2XLJ8
zHJlOL2L
ZghxMBly
RVnyARBM
simZwI6Z
hRp1LhQE
mNlhfe1A
QiW7kLYI
ceAoXQxx
1qnSNFC7
49mBDfCm
2hqduMgW
umT8EIXw
EzVNMZcU
MyG354HP
KiVBdTMX
0Dnos2gk
Il56HHo7
8MMJa6c4
ERNIhSWk
1MXraiz9
h3c46AVo
Gxut53XS
icvgFxWE
asyz1jPE
Hrr5rNlR
qSFKYq5J
jsbvnGJJ
bj1MDfYI
KzHbwlzn
4KCF9Y7w
NKoYHz73
LC3wN8lP
fak0RT9a
WtUrJr11
RI6uVUzh
6UmAiL2H
GMVFisvA
VF3DStg6
eaUWKK5D
wRTJRY3g
l1ghtQCD
dViuNVsj
w

BBfXCvNe
OmzDQa7S
b8WW8KMc
f4vQHYIZ
wGgYQCzL
rK8QDVof
Q2DoUVCL
5gZtl4bL
XlQGDrdb
ohcusYZc
7qj3UrkU
chsBNGmJ
7OHtcn1Z
15lEpldE
LMEKfsPt
8BLeYNSO
t5TLvGes
AtgT369E
6FmfhJxL
ruixXXJs
isS6IG39
fO4nroL9
8c1sF2CM
wX3EQAPu
rVHJzpo8
LtakuRwC
yPT5JaJx
dv89jgip
uhsgwEWX
qt1farO7
3jaOrAPh
2Oki1TZT
Ry7WqOgZ
Z7JqMQzu
9MpjI8xF
b5lyUYiv
kmWD4yQ1
CFPz3QHX
Gxywco2J
Day1CzPI
1jx2clzD
XhHdbRd8
IhBXD4Zz
sB44qbFF
4tS7nXwb
GJU484mN
kr18QbLR
A0A0KuQg
7VLHXcZ0
KkhmTcSr
vG8NwlWe
8jpzMZBw
SDPDt6r2
NdQPK9LX
zP8SQqB8
GRFgoAT6
acfjZIi9
H0RB60vw
XcT8WL1F
mr1kSRV2
7llidxIH
SeiZHaJr
q7Gy5nXc
dMmp24fr
UYQWYvfo
GHGiIc06
KZKSnfy7
GhA9kdBt
K6dyJ2Mw
1MpZCMRs
oc5Okgjq
473wOdqi
Jwm7gBCz
ox3EjEhP
RcWTw0K5
wNWr72u6
GOjnobZO
amtvCHl6
gdsmyV4C
1ScXYy3r
dK46E094
7Bvs8vKk
N4vflBwr
BnNOunMa
Y4LJ74lA
3QG6DxRg
CFpJNW7e
J07srTcU
YlXZMuXV
jZSyDeNl
28SlIrvt
WF8oMrhP
PpZDXyI2
3iZYUraX
vYh6Kl6A
WZUrZAm1
5uaDseel
9QzIAZmX
KpDXD1HJ
9Z1f8nax
RYEPFTzV
D33bTcwN
7aQjoFHt
J6oQBI88
e9zUgTez
pBKSHBHv
28IaxX3k
EJTb5mEo
znlCXjVK
rrr4pFrh
I75jRlwh
2

KeyboardInterrupt: 

In [118]:
MOVES_TRANS = concat(transposed_moves)

1          2          3          4          5          6   \
TZJHLljE  123442959  123717766  123297963  123842441  259080238  124130989   

                 7          8          9          10         11         12  \
TZJHLljE  259638457  259924149  186302529  237858084  185871841  183722274   

                 13  
TZJHLljE  184172591

In [ ]:
MOVES_TRANS.to_csv('./moves_trans.csv')

# @@@ Cell > Run All Above @@@

In [ ]:
games_s['moves_class'].

In [ ]:
games[['moves_win','winner_bin']]

## UMAP model

---

Parameters:

In [ ]:
distance_metric = 'euclidean'
# metrics_available = euclidean, manhattan, chebyshev, minkowski, canberra, braycurtis, mahalanobis,
# wminkowski, seuclidean, cosine, correlation, haversine, hamming, jaccard, dice, russelrao, kulsinski,
# ll_dirichlet, hellinger, rogerstanimoto, sokalmichener, sokalsneath, yule

n_neighbors = 15
min_dist = 0.2

Instatiate & train the model:

In [ ]:
## Instantiate UMAP model
reducer = umap.UMAP(metric=distance_metric,
                   n_neighbors=n_neighbors,
                   min_dist=min_dist)

In [ ]:
## Train the model
embedding = reducer.fit_transform(hot_df)

---

## Plot

---

Parameters:

In [ ]:
## switches
cbar_switch = True

## display
alpha = 1
marker_shape = 'o'
marker_size = 2
dpi = 300

## data
cmap_variable = games['game_len_mins']

## colors
facecolor = 'black'
labelcolor = 'white'
cmap = 'viridis'

Plot script:

In [ ]:
## Render vis
fig = plt.figure(figsize=(14,12))
fig.patch.set_facecolor(facecolor)

plot = plt.scatter(embedding[:,0], 
                embedding[:,1], 
                marker=marker_shape,
                s=marker_size,
                edgecolor='none',
                c=cmap_variable,
                cmap='viridis', 
                alpha=alpha)

plt.axis("off")
plt.title(f'20k chess games; distance={distance_metric}; nn={n_neighbors}, min=dist={min_dist}', 
          color='w')

if cbar_switch:
    cbar = plt.colorbar(fraction=0.05, 
                    shrink=0.5,
                    ticks=[games['game_len_mins'].min(), 
                        games['game_len_mins'].quantile(q=0.50),
                        games['game_len_mins'].quantile(q=0.75),
                        games['game_len_mins'].max()], alpha=alpha
                           )

    cbar.set_label(cmap_variable.name, color=labelcolor)
    cbar.ax.yaxis.set_tick_params(color=labelcolor)
    cbar.solids.set_edgecolor("face")

plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color=labelcolor)

_ = plot

# plt.savefig(f"screens/{distance_metric}_nn{n_neighbors}_mind{min_dist}_v{notebook_ver}.png", 
#             dpi=dpi, 
#             facecolor=facecolor)